In [1]:
# Import Libraries
import time
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

In [2]:
# Disable webpage and Image Loading

# Firefox
options = webdriver.FirefoxOptions()
options.add_argument("--headless")  # Enable headless mode
options.set_preference("permissions.default.image", 2)  # Disable images loading
driver = webdriver.Firefox(options=options)

# chrome
#options = webdriver.ChromeOptions()
#options.add_argument("--headless=new")
#options.add_argument('--blink-settings=imagesEnabled=false')
#driver = webdriver.Chrome(options=options)

In [3]:
# Navigate to Google Search
search_keyword = "pisco"
driver.get("https://www.santaisabel.cl/busqueda?ft=" + search_keyword)

In [4]:
# Define the number of times to scroll
scroll_count = 5

# Simulate continuous scrolling using JavaScript
for _ in range(scroll_count):
   driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
   time.sleep(2)  # Wait for the new results to load (adjust as needed)

# Get the page source after scrolling
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# search_results
search_results = soup.find_all("div", class_="product-card-wrap")

# close webdriver
driver.quit()

In [11]:
data_list = []

for result in search_results:
    # Verificar si el producto está fuera de stock
    out_of_stock_flag = result.find("div", class_="product-card-flag out-of-stock")
    if out_of_stock_flag:
        continue  # Si el producto está fuera de stock, pasar al siguiente resultado
    
    product_info = {}
    
    # Obtener información de cada elemento
    main_price_tag = result.find("span", class_="prices-main-price")
    main_price = main_price_tag.text.strip()
    main_price_value = int(main_price.replace('$', '').replace('.', ''))

    # Agregar la información al diccionario
    product_info["main_price"] = main_price_value
    
    # Verificar si existe "prices-old-price"
    old_price_tag = result.find("span", class_="prices-old-price")
    if old_price_tag:
        old_price = old_price_tag.text.strip()

        # Calcular el porcentaje de descuento y agregar al diccionario
        old_price_value = int(old_price.replace('$', '').replace('.', ''))
        discount_percentage = round(((old_price_value - main_price_value) / old_price_value) * 100)
        
        # Agregar la información al diccionario
        product_info["old_price"] = old_price_value
        product_info["discount"] = discount_percentage
    else:
        # Agregar la información al diccionario
        product_info["old_price"] = None
        product_info["discount"] = None
        
    brand = result.find("a", class_="product-card-brand").text.strip()
    name = result.find("a", class_="product-card-name").text.strip()
    
    # Agregar la información al diccionario
    product_info["brand"] = brand
    product_info["name"] = name
    
    # Agregar el diccionario a la lista
    data_list.append(product_info)

In [12]:
# Crear un DataFrame de Pandas a partir de la lista de diccionarios
df = pd.DataFrame(data_list)
df.sort_values(by=['discount'], ascending=False, inplace=True)
# Imprimir el DataFrame
df.head(20)

,main_price,old_price,discount,brand,name
3,6250,8690.0,28.0,Tres Erres,Pisco 3R Doble Destilado 35° 1 L
1,6690,8490.0,21.0,Mistral,Pisco Añejado en Roble 35° botella 750 cc
14,7950,9890.0,20.0,Mistral,Pisco Añejado en Roble 46° botella 750 cc
21,9250,11590.0,20.0,Horcón Quemado,Pisco Horcón Quemado 645 cc 35° Artesanal añejado
5,11990,14890.0,19.0,Mistral,Pisco Mistral Nobel Apple 35° 750 cc
13,11990,14890.0,19.0,Mistral,Pisco Mistral Nobel Honey 35° 750 cc
39,11990,14890.0,19.0,Mistral,Pisco Nobel 30° Fire botella 750 cc
15,24990,30590.0,18.0,Mistral,Pisco Gran Nobel Reserva Privada 40° botella 7...
16,7750,9490.0,18.0,Mistral,Pisco Añejado en Roble 40° botella 750 cc
10,4190,4790.0,13.0,Campanario,Pisco 35° 700 cc
